In [1]:
from sklearn.linear_model import LogisticRegression as LR
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from vocabulary import Vocabulary
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV  # 带交叉验证的网格搜索

import pandas as pd
import numpy as np
import pickle
import warnings

warnings.filterwarnings("ignore")

## 1、特征为residue_vocabulary，其序列残基对应词汇表所做的字符编码

In [2]:
# 将序列残基对应词典转换成特征
data = pd.read_excel('/home/dldx/R-H/code/classification/ML/data/mafft_MSA_R_H.xlsx')

vocabulary = Vocabulary.get_vocabulary_from_sequences(data.Sequence.values)  #将蛋白质序列构造成词典

In [3]:
input_seq_tensor = []
input_lable = []
for i in range(179):
    # 将序列残基进行字符编码
    seq_tensor = vocabulary.seq_to_tensor(data.Sequence[i])
    input_seq_tensor.append(np.array(seq_tensor))
    
input_seq_tensor = np.array(input_seq_tensor)
input_lable = np.array(data.Tier)

In [13]:
# 设置超参数
param_dict = { "penalty": ["l1","l2"], "C": np.linspace(0.05,1,19), "random_state":np.arange(0, 31, 3), "max_iter": [100, 500, 1000]}

for r in range(0,31):
    # train_test_split 按比例划分原数据集
    Xtrain,Xtest,Ytrain,Ytest = train_test_split(input_seq_tensor,input_lable,train_size=0.8,random_state = r)

    # 建立逻辑回归模型
    clf = LR()
    # 将模型加入超参数网格搜索和交叉验证     
    clf = GridSearchCV(clf, param_grid=param_dict, cv=5)
    # 模型训练
    clf = clf.fit(Xtrain,Ytrain)
    # 模型评估
    Y_pred = clf.predict(Xtest)
    cm = confusion_matrix(Ytest,Y_pred)
    tn, tp, fn, fp = cm[0][0], cm[1][1], cm[1][0], cm[0][1]
    n = tp + fp + tn + fn
    accuracy = (tp + tn)/n 
    mcc = ((tp*tn) - (fp*fn))/np.sqrt((tp+fp)*(tn+fn)*(tp+fn)*(tn+fp))
    sens = tp/(tp + fn) * 100 if tp + fp != 0 else 0
    spec = tn/(tn + fp) * 100 if tn + fn != 0 else 0

    print("数据集划分 random_state：" + str(r)  + "时，")
    print("Evaluate：")
    print("  accuracy : " + str(accuracy))
    print("  mcc : " + str(mcc))
    print("  sens : " + str(sens))
    print("  spec : " + str(spec)) 
    print("============================")

数据集划分 random_state：0时，
Evaluate：
  accuracy : 0.7777777777777778
  mcc : 0.577519996976099
  sens : 66.66666666666666
  spec : 85.71428571428571
数据集划分 random_state：1时，
Evaluate：
  accuracy : 0.7222222222222222
  mcc : 0.4705886038624533
  sens : 64.70588235294117
  spec : 78.94736842105263
数据集划分 random_state：2时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : 0.7427813527082074
  sens : 35.0
  spec : 100.0
数据集划分 random_state：3时，
Evaluate：
  accuracy : 0.7777777777777778
  mcc : 0.5270462766947298
  sens : 81.25
  spec : 75.0
数据集划分 random_state：4时，
Evaluate：
  accuracy : 0.75
  mcc : 0.4136725758374611
  sens : 81.81818181818183
  spec : 72.0
数据集划分 random_state：5时，
Evaluate：
  accuracy : 0.75
  mcc : 0.5558842981603065
  sens : 68.42105263157895
  spec : 82.35294117647058
数据集划分 random_state：6时，
Evaluate：
  accuracy : 0.5555555555555556
  mcc : 0.11726039399558573
  sens : 64.28571428571429
  spec : 50.0
数据集划分 random_state：7时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc : 0.1956998

## 2、特征为residue_disorder，其序列残基对应disorder所做的字符编码

In [5]:
data1 = pd.read_csv('/home/dldx/R-H/code/classification/ML/data/disorder/MSA_disorder_R_H.csv')

In [6]:
input_feature = pickle.load(open('/home/dldx/R-H/code/classification/ML/data/disorder/all_seq_disorder_R_H.pkl','rb'))
input_lable1 = np.array(data1.Tier)

In [17]:
# 设置超参数
param_dict = { "penalty": ["l1","l2"], "C": np.linspace(0.05,1,19), "random_state":np.arange(0, 31, 3), "max_iter": [100, 500, 1000]}

for r in range(0,31):
    # train_test_split 按比例划分原数据集
    Xtrain,Xtest,Ytrain,Ytest = train_test_split(input_feature,input_lable1,train_size=0.8,random_state = r)

    # 建立逻辑回归模型
    clf = LR()
    # 将模型加入超参数网格搜索和交叉验证     
    clf = GridSearchCV(clf, param_grid=param_dict, cv=5)
    # 模型训练
    clf = clf.fit(Xtrain,Ytrain)
    # 模型评估
    Y_pred = clf.predict(Xtest)
    cm = confusion_matrix(Ytest,Y_pred)
    tn, tp, fn, fp = cm[0][0], cm[1][1], cm[1][0], cm[0][1]
    n = tp + fp + tn + fn
    accuracy = (tp + tn)/n 
    mcc = ((tp*tn) - (fp*fn))/np.sqrt((tp+fp)*(tn+fn)*(tp+fn)*(tn+fp))
    sens = tp/(tp + fn) * 100 if tp + fp != 0 else 0
    spec = tn/(tn + fp) * 100 if tn + fn != 0 else 0

    print("数据集划分 random_state：" + str(r)  + "时，")
    print("Evaluate：")
    print("  accuracy : " + str(accuracy))
    print("  mcc : " + str(mcc))
    print("  sens : " + str(sens))
    print("  spec : " + str(spec)) 
    print("============================")

数据集划分 random_state：0时，
Evaluate：
  accuracy : 0.8055555555555556
  mcc : 0.7446512569689885
  sens : 60.0
  spec : 95.23809523809523
数据集划分 random_state：1时，
Evaluate：
  accuracy : 0.6944444444444444
  mcc : 0.46829290579084704
  sens : 52.94117647058824
  spec : 84.21052631578947
数据集划分 random_state：2时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc : 0.6079600189283705
  sens : 45.0
  spec : 93.75
数据集划分 random_state：3时，
Evaluate：
  accuracy : 0.6944444444444444
  mcc : 0.39036002917941326
  sens : 62.5
  spec : 75.0
数据集划分 random_state：4时，
Evaluate：
  accuracy : 0.8055555555555556
  mcc : 0.5177731682921968
  sens : 81.81818181818183
  spec : 80.0
数据集划分 random_state：5时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc : 0.6067880364121376
  sens : 42.10526315789473
  spec : 94.11764705882352
数据集划分 random_state：6时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : 0.1556997888323046
  sens : 21.428571428571427
  spec : 86.36363636363636
数据集划分 random_state：7时，
Evaluate：
  accuracy : 0.5555

## 3、特征为21个无规则卷曲的关键区域长度，IDR1, IDR2, IDR3, IDR4, IDR5, IDR6, IDR7, IDR8,IDR9,IDR10,IDR11,IDR12,IDR13,IDR14,IDR15,IDR16,IDR17,IDR18,IDR19,IDR20,IDR21。

In [4]:
data2_R = pickle.load(open('/www/yang/csv/align/count_C_pickle/mafft_MSA/end_count_C_R_1.pkl','rb'))
data2_H = pickle.load(open('/www/yang/csv/align/count_C_pickle/mafft_MSA/end_count_C_H_3.pkl','rb'))

In [6]:
data2_R[65][15]

3

In [5]:
all_21_IDR = []
for position in range(21):
    one_IDR = []
    R_IDR = [i[position] for i in data2_R]
    H_IDR = [i[position] for i in data2_H]
 
    # 转化为二维数组
    for i in range(72):
        test =[]
        test.append(R_IDR[i])
        one_IDR.append(test)
    for i in range(107):
        test =[]
        test.append(H_IDR[i])
        one_IDR.append(test)

    # 所有IDR添加到一个数组
    all_21_IDR.append(one_IDR)

In [18]:
# 设置超参数
param_dict = { "penalty": ["l1","l2"], "C": np.linspace(0.05,1,19), "random_state":np.arange(0, 31, 3), "max_iter": [100, 500, 1000]}
# 21个IDR区域分别进行分类
for position in range(21):
    for r in range(0,31):
        # train_test_split 按比例划分原数据集
        Xtrain,Xtest,Ytrain,Ytest = train_test_split(all_21_IDR[position],input_lable,train_size=0.8,random_state = r)

        # 建立逻辑回归模型
        clf = LR()
        # 将模型加入超参数网格搜索和交叉验证     
        clf = GridSearchCV(clf, param_grid=param_dict, cv=5)
        # 模型训练
        clf = clf.fit(Xtrain,Ytrain)
        # 模型评估
        Y_pred = clf.predict(Xtest)
        cm = confusion_matrix(Ytest,Y_pred)
        tn, tp, fn, fp = cm[0][0], cm[1][1], cm[1][0], cm[0][1]
        n = tp + fp + tn + fn
        accuracy = (tp + tn)/n 
        mcc = ((tp*tn) - (fp*fn))/np.sqrt((tp+fp)*(tn+fn)*(tp+fn)*(tn+fp))
        sens = tp/(tp + fn) * 100 if tp + fp != 0 else 0
        spec = tn/(tn + fp) * 100 if tn + fn != 0 else 0

        print("数据集划分 random_state：" + str(r)  + "时，")
        print("Evaluate：")
        print("  accuracy : " + str(accuracy))
        print("  mcc : " + str(mcc))
        print("  sens : " + str(sens))
        print("  spec : " + str(spec)) 
        print("============================")
    print("以上为IDR__" + str(position + 1) + "时，evaluation")
    print("***********************************************************************")

数据集划分 random_state：0时，
Evaluate：
  accuracy : 0.5833333333333334
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：1时，
Evaluate：
  accuracy : 0.5277777777777778
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：2时，
Evaluate：
  accuracy : 0.4444444444444444
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：3时，
Evaluate：
  accuracy : 0.5555555555555556
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：4时，
Evaluate：
  accuracy : 0.6944444444444444
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：5时，
Evaluate：
  accuracy : 0.4722222222222222
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：6时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：7时，
Evaluate：
  accuracy : 0.7777777777777778
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：8时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：9时，
Evaluate：
  accuracy : 0.4722222222222222
  mcc : 

数据集划分 random_state：30时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
以上为IDR__2时，evaluation
***********************************************************************
数据集划分 random_state：0时，
Evaluate：
  accuracy : 0.5833333333333334
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：1时，
Evaluate：
  accuracy : 0.5277777777777778
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：2时，
Evaluate：
  accuracy : 0.4444444444444444
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：3时，
Evaluate：
  accuracy : 0.5555555555555556
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：4时，
Evaluate：
  accuracy : 0.6944444444444444
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：5时，
Evaluate：
  accuracy : 0.4722222222222222
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：6时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：7时，
Evaluate：
  accuracy : 0.7222222222222222
  mcc : -0.25928

数据集划分 random_state：25时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：26时，
Evaluate：
  accuracy : 0.5833333333333334
  mcc : -0.06497983230676314
  sens : 16.666666666666664
  spec : 79.16666666666666
数据集划分 random_state：27时，
Evaluate：
  accuracy : 0.7222222222222222
  mcc : 0.03340765523905305
  sens : 12.5
  spec : 89.28571428571429
数据集划分 random_state：28时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：29时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：30时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
以上为IDR__4时，evaluation
***********************************************************************
数据集划分 random_state：0时，
Evaluate：
  accuracy : 0.5277777777777778
  mcc : -0.11952286093343938
  sens : 13.333333333333334
  spec : 80.95238095238095
数据集划分 random_state：1时，
Evaluate：
  accuracy : 0.58333333

数据集划分 random_state：15时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：16时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：17时，
Evaluate：
  accuracy : 0.7222222222222222
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：18时，
Evaluate：
  accuracy : 0.3888888888888889
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：19时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : -0.4581897949526569
  sens : 0.0
  spec : 95.65217391304348
数据集划分 random_state：20时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：21时，
Evaluate：
  accuracy : 0.5
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：22时，
Evaluate：
  accuracy : 0.6944444444444444
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：23时，
Evaluate：
  accuracy : 0.6944444444444444
  mcc : 0.816496580927726
  sens : 21.428571428571427
  spec : 100.0
数据集划分 random_state

数据集划分 random_state：14时，
Evaluate：
  accuracy : 0.5833333333333334
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：15时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：16时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：17时，
Evaluate：
  accuracy : 0.7222222222222222
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：18时，
Evaluate：
  accuracy : 0.3888888888888889
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：19时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：20时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：21时，
Evaluate：
  accuracy : 0.5
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：22时，
Evaluate：
  accuracy : 0.6944444444444444
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：23时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
 

...

数据集划分 random_state：27时，
Evaluate：
  accuracy : 0.8055555555555556
  mcc : 0.5263613559678152
  sens : 25.0
  spec : 96.42857142857143
数据集划分 random_state：28时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : -0.4581897949526569
  sens : 0.0
  spec : 95.65217391304348
数据集划分 random_state：29时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：30时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
以上为IDR__13时，evaluation
***********************************************************************
数据集划分 random_state：0时，
Evaluate：
  accuracy : 0.5833333333333334
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：1时，
Evaluate：
  accuracy : 0.5277777777777778
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：2时，
Evaluate：
  accuracy : 0.4444444444444444
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：3时，
Evaluate：
  accuracy : 0.5555555555555556
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_stat

数据集划分 random_state：22时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc : -0.3718678720805473
  sens : 0.0
  spec : 96.0
数据集划分 random_state：23时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：24时，
Evaluate：
  accuracy : 0.5833333333333334
  mcc : -0.504524979109513
  sens : 0.0
  spec : 95.45454545454545
数据集划分 random_state：25时，
Evaluate：
  accuracy : 0.5833333333333334
  mcc : -0.504524979109513
  sens : 0.0
  spec : 95.45454545454545
数据集划分 random_state：26时，
Evaluate：
  accuracy : 0.6944444444444444
  mcc : 0.8280786712108251
  sens : 8.333333333333332
  spec : 100.0
数据集划分 random_state：27时，
Evaluate：
  accuracy : 0.8333333333333334
  mcc : 0.634745449542008
  sens : 37.5
  spec : 96.42857142857143
数据集划分 random_state：28时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc : 0.3992747047523452
  sens : 15.384615384615385
  spec : 95.65217391304348
数据集划分 random_state：29时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : 0.7928249671720918


数据集划分 random_state：16时，
Evaluate：
  accuracy : 0.5833333333333334
  mcc : -0.14744195615489714
  sens : 7.6923076923076925
  spec : 86.95652173913044
数据集划分 random_state：17时，
Evaluate：
  accuracy : 0.4166666666666667
  mcc : -0.29568814265498417
  sens : 10.0
  spec : 53.84615384615385
数据集划分 random_state：18时，
Evaluate：
  accuracy : 0.4166666666666667
  mcc : 0.09304842103984708
  sens : 9.090909090909092
  spec : 92.85714285714286
数据集划分 random_state：19时，
Evaluate：
  accuracy : 0.5833333333333334
  mcc : 0.019702760155977515
  sens : 23.076923076923077
  spec : 78.26086956521739
数据集划分 random_state：20时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : 0.08798826901281197
  sens : 16.666666666666664
  spec : 87.5
数据集划分 random_state：21时，
Evaluate：
  accuracy : 0.5277777777777778
  mcc : 0.7171371656006362
  sens : 5.555555555555555
  spec : 100.0
数据集划分 random_state：22时，
Evaluate：
  accuracy : 0.5833333333333334
  mcc : -0.07559289460184544
  sens : 18.181818181818183
  spec : 76.0
数据集划分 ra

数据集划分 random_state：10时，
Evaluate：
  accuracy : 0.5555555555555556
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：11时，
Evaluate：
  accuracy : 0.5
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：12时，
Evaluate：
  accuracy : 0.5277777777777778
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：13时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：14时，
Evaluate：
  accuracy : 0.5833333333333334
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：15时，
Evaluate：
  accuracy : 0.6666666666666666
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：16时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：17时，
Evaluate：
  accuracy : 0.7222222222222222
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：18时，
Evaluate：
  accuracy : 0.3888888888888889
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：19时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : nan
 

数据集划分 random_state：4时，
Evaluate：
  accuracy : 0.6944444444444444
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：5时，
Evaluate：
  accuracy : 0.4722222222222222
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：6时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：7时，
Evaluate：
  accuracy : 0.7777777777777778
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：8时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：9时，
Evaluate：
  accuracy : 0.4722222222222222
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：10时，
Evaluate：
  accuracy : 0.5555555555555556
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：11时，
Evaluate：
  accuracy : 0.5
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：12时，
Evaluate：
  accuracy : 0.5277777777777778
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：13时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens 

In [7]:
# 重新跑关于IDR16由数据更改的情况
# 设置超参数
param_dict = { "penalty": ["l1","l2"], "C": np.linspace(0.05,1,19), "random_state":np.arange(0, 31, 3), "max_iter": [100, 500, 1000]}
# 21个IDR区域分别进行分类
for r in range(0,31):
    # train_test_split 按比例划分原数据集
    Xtrain,Xtest,Ytrain,Ytest = train_test_split(all_21_IDR[15],input_lable,train_size=0.8,random_state = r)

    # 建立逻辑回归模型
    clf = LR()
    # 将模型加入超参数网格搜索和交叉验证     
    clf = GridSearchCV(clf, param_grid=param_dict, cv=5)
    # 模型训练
    clf = clf.fit(Xtrain,Ytrain)
    # 模型评估
    Y_pred = clf.predict(Xtest)
    cm = confusion_matrix(Ytest,Y_pred)
    tn, tp, fn, fp = cm[0][0], cm[1][1], cm[1][0], cm[0][1]
    n = tp + fp + tn + fn
    accuracy = (tp + tn)/n 
    mcc = ((tp*tn) - (fp*fn))/np.sqrt((tp+fp)*(tn+fn)*(tp+fn)*(tn+fp))
    sens = tp/(tp + fn) * 100 if tp + fp != 0 else 0
    spec = tn/(tn + fp) * 100 if tn + fn != 0 else 0

    print("数据集划分 random_state：" + str(r)  + "时，")
    print("Evaluate：")
    print("  accuracy : " + str(accuracy))
    print("  mcc : " + str(mcc))
    print("  sens : " + str(sens))
    print("  spec : " + str(spec)) 
    print("============================")
print("以上为IDR__" + str(16) + "时，evaluation")

数据集划分 random_state：0时，
Evaluate：
  accuracy : 0.5833333333333334
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：1时，
Evaluate：
  accuracy : 0.5277777777777778
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：2时，
Evaluate：
  accuracy : 0.4444444444444444
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：3时，
Evaluate：
  accuracy : 0.5555555555555556
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：4时，
Evaluate：
  accuracy : 0.6944444444444444
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：5时，
Evaluate：
  accuracy : 0.4722222222222222
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：6时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：7时，
Evaluate：
  accuracy : 0.7777777777777778
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：8时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : nan
  sens : 0
  spec : 100.0
数据集划分 random_state：9时，
Evaluate：
  accuracy : 0.4722222222222222
  mcc : 

In [8]:
# 将21个区域的无规则卷曲长度一起作为特征
all_R_H_IDR =[]
all_R_H_IDR.extend(data2_R)
all_R_H_IDR.extend(data2_H)

# 设置超参数
param_dict = { "penalty": ["l1","l2"], "C": np.linspace(0.05,1,19), "random_state":np.arange(0, 31, 3), "max_iter": [100, 500, 1000]}

for r in range(0,31):
    # train_test_split 按比例划分原数据集
    Xtrain,Xtest,Ytrain,Ytest = train_test_split(all_R_H_IDR,input_lable,train_size=0.8,random_state = r)

    # 建立逻辑回归模型
    clf = LR()
    # 将模型加入超参数网格搜索和交叉验证     
    clf = GridSearchCV(clf, param_grid=param_dict, cv=5)
    # 模型训练
    clf = clf.fit(Xtrain,Ytrain)
    # 模型评估
    Y_pred = clf.predict(Xtest)
    cm = confusion_matrix(Ytest,Y_pred)
    tn, tp, fn, fp = cm[0][0], cm[1][1], cm[1][0], cm[0][1]
    n = tp + fp + tn + fn
    accuracy = (tp + tn)/n 
    mcc = ((tp*tn) - (fp*fn))/np.sqrt((tp+fp)*(tn+fn)*(tp+fn)*(tn+fp))
    sens = tp/(tp + fn) * 100 if tp + fp != 0 else 0
    spec = tn/(tn + fp) * 100 if tn + fn != 0 else 0

    print("数据集划分 random_state：" + str(r)  + "时，")
    print("Evaluate：")
    print("  accuracy : " + str(accuracy))
    print("  mcc : " + str(mcc))
    print("  sens : " + str(sens))
    print("  spec : " + str(spec)) 
    print("============================")

数据集划分 random_state：0时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : 0.4629100498862757
  sens : 20.0
  spec : 95.23809523809523
数据集划分 random_state：1时，
Evaluate：
  accuracy : 0.5833333333333334
  mcc : 0.18731716231633883
  sens : 41.17647058823529
  spec : 73.68421052631578
数据集划分 random_state：2时，
Evaluate：
  accuracy : 0.6111111111111112
  mcc : 0.7302967433402215
  sens : 30.0
  spec : 100.0
数据集划分 random_state：3时，
Evaluate：
  accuracy : 0.5555555555555556
  mcc : 0.11180339887498948
  sens : 62.5
  spec : 50.0
数据集划分 random_state：4时，
Evaluate：
  accuracy : 0.6944444444444444
  mcc : 0.2984810028978546
  sens : 63.63636363636363
  spec : 72.0
数据集划分 random_state：5时，
Evaluate：
  accuracy : 0.5555555555555556
  mcc : 0.23338001400466832
  sens : 31.57894736842105
  spec : 82.35294117647058
数据集划分 random_state：6时，
Evaluate：
  accuracy : 0.6388888888888888
  mcc : 0.2910190408056438
  sens : 21.428571428571427
  spec : 90.9090909090909
数据集划分 random_state：7时，
Evaluate：
  accuracy : 0.5833